<a href="https://colab.research.google.com/github/AI4GoodE1/AI4GoodE1/blob/main/distilBERTmodelEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

#imports
import numpy as np
import pandas as pd
# import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split

#transformers import
from transformers import DistilBertTokenizerFast, DistilBertModel
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from sklearn.metrics import confusion_matrix

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
trained_model = TFDistilBertForSequenceClassification.from_pretrained("/content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model")
#trained_model = tf.keras.models.load_model("/content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model/config.json")


Some layers from the model checkpoint at /content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/Shareddrives/AI4GoodE1 - Scam Detection Project/trained_distilbert_model and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a d

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [4]:
test_email1 = "Hi team, let’s use my webex for our meeting at 3:15 again. Here is the link: https://rbcteams.webex.com/meet/test.test Thanks"
test_email1 = tokenizer(test_email1, truncation=True, padding=True, return_tensors="tf")
print(test_email1)
output = trained_model(test_email1)
print(output.loss)
print(f"trained: {output.logits}")

{'input_ids': <tf.Tensor: shape=(1, 46), dtype=int32, numpy=
array([[  101,  7632,  2136,  1010,  2292,  1521,  1055,  2224,  2026,
         4773, 10288,  2005,  2256,  3116,  2012,  1017,  1024,  2321,
         2153,  1012,  2182,  2003,  1996,  4957,  1024, 16770,  1024,
         1013,  1013, 21144,  6593,  5243,  5244,  1012,  4773, 10288,
         1012,  4012,  1013,  3113,  1013,  3231,  1012,  3231,  4283,
          102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 46), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]], dtype=int32)>}
None
trained: [[ 3.7335205 -3.8806746]]


In [5]:
# Get data
df = pd.read_csv('https://raw.githubusercontent.com/AI4GoodE1/AI4GoodE1/main/fraud_email_preprocessed.csv')
X = list(df['Text'])
y = list(df['Class'])
X_Train, X_Test, y_Train, y_Test = train_test_split(X,y,test_size=0.3,random_state=0)
train_encodings = tokenizer(X_Train, truncation=True, padding=True) #, return_tensors="tf")
test_encodings = tokenizer(X_Test, truncation=True, padding=True) #, return_tensors="tf")

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_Train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_Test
))

In [6]:
# setting most of these features to one further reduces runtime
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=4,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    eval_steps = 10
)

with training_args.strategy.scope():
    model = trained_model

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

/usr/local/lib/python3.7/dist-packages/transformers/trainer_tf.py:120: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  FutureWarning,


In [8]:
output=trainer.predict(test_dataset)[1]

In [9]:
confusionMatrixResult=confusion_matrix(y_Test,output)
tn, fp, fn, tp = confusionMatrixResult.ravel()
print("Confusion Matrix results:")
print(confusionMatrixResult)
print(f"Number of Correctly Predicted Fraud Emails: {tp}")
print(f"Number of Falsely Predicted Fraud Emails: {fp}")
print(f"Number of Correctly Predicted Legit Emails: {tn}")
print(f"Number of Falsely Predicted Fraud Emails: {fn}")

Confusion Matrix results:
[[1775    0]
 [   0 1300]]
Number of Correctly Predicted Fraud Emails: 1300
Number of Falsely Predicted Fraud Emails: 0
Number of Correctly Predicted Legit Emails: 1775
Number of Falsely Predicted Fraud Emails: 0
